In [ ]:
from faker import Faker
fake = Faker('pt_BR')
import datetime, json, requests, asyncio, random
from dateutil.relativedelta import relativedelta
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData

HOST = 'https://padariacasedb.documents.azure.com:443/'
MASTER_KEY = 'KPxz6QHRsC0rnSC0A2Wi4C2PNVapVy0JU9GpCyUC4LRrD9Xqrmh18BCShyvXTHOrUxyNIFEf848tHGmv221eGA=='
DATABASE_ID = 'padariadb'
CONTAINER_ID = 'customer'

In [ ]:
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
db = client.create_database_if_not_exists(id=DATABASE_ID)
container = db.create_container_if_not_exists(
    id=CONTAINER_ID, 
    partition_key=PartitionKey(path="/cpf"),
)

In [ ]:
# enable_cross_partition_query should be set to True as the container is partitioned
items = list(container.query_items(
    query="SELECT top 1 c.id as customerId,c.originBranchId as branchId,c.cpf,c.name FROM customer c where RAND() <= @id",
    parameters=[
        { "name":"@id", "value": .3 }
    ],
    enable_cross_partition_query=True
))

print('Item queried by Id {0}'.format(items[0].get("customerId")))
print('Item queried by Id {0}'.format(items[0].get("cpf")))
print('Item queried by Id {0}'.format(items[0].get("branchId")))
print('Item queried by Id {0}'.format(items[0].get("name")))

In [ ]:
Turnstile = {
    "id": fake.hexify(text='^^^^-^^^^-^^^^-^^^^'),
    "branchId": fake.random_number(3),
    "customerId": random.randint(0,1000), 
    "type": 1,
    "cardID": random.randint(0,100),
    "dtLog": datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
    "products": [
      {
        "productId": 0,
        "quantity": 0,
        "price": 0
      }
    ]
  }

print(Turnstile)


In [ ]:
async def run():
  producer = EventHubProducerClient.from_connection_string(conn_str="Endpoint=sb://padaria-eventhubs.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=OW1DDFjQvnKNb81LzhcQxF9BRwotHPC73y09ffl+6BI=", eventhub_name="turnstile-topic")
  async with producer:
      # Create a batch.
      event_data_batch = await producer.create_batch()

      # Add events to the batch.
      event_data_batch.add(EventData(json.dumps(Turnstile, indent=4)))

      # Send the batch of events to the event hub.
      await producer.send_batch(event_data_batch)
      print("sent event")

loop = asyncio.get_event_loop()
loop.run_until_complete(run())